# Preprocess

Preprocess the raw results from OntoGPT

Draft: https://docs.google.com/document/d/1H103ux6Dd1_bPM0un4RwutBLcYJx-0ybil2AwlAvG_Q/edit#

## Initial setup

Import libraries, create a GO adapter (for calculating closures/ancestors)

In [1]:
# note the gpt4 dir includes combined results from davinci, 3.5 and 4
results_dir = "results/human/gpt4"

In [2]:
import yaml
from yaml import Loader
from collections import defaultdict

import pandas as pd
import math

from oaklib import get_adapter
from oaklib.datamodels.vocabulary import IS_A, PART_OF
from ontogpt.evaluation.enrichment.eval_enrichment import EvalEnrichment
go = get_adapter("sqlite:obo:go")

In [3]:
# Load the closure/ancestor map
# (takes a minute)
closure_map = defaultdict(set)
for s, _, o in go.relationships(predicates=[IS_A, PART_OF], include_entailed=True):
    closure_map[s].add(o)

In [4]:
print(len(closure_map))

84220


In [5]:
# ruamel is faster than pyyaml
from ruamel.yaml import YAML
ryaml = YAML()

## Load YAML results

See OntoGPT for details of data model

These results are nested, with a parent gene set object containing all runs.
We want to flatten this for analysis

In [6]:
from ontogpt.evaluation.enrichment.eval_enrichment import GeneSetComparison

In [7]:
# assumes comparisons have been run and concatenated (see Makefile) 
import glob
def load_gene_set_results():
    results = []
    for fn in glob.glob(f"{results_dir}/*.yaml"):
        print(fn)
        with open(fn) as f:
            #obj = yaml.load(f, Loader)
            #obj = yaml.safe_load(f)
            obj = ryaml.load(f)
            results.extend(obj)
    return results

In [8]:
comps = load_gene_set_results()

results/human/gpt4/canonical-glycolysis-gocam-results-2.yaml
results/human/gpt4/bicluster_RNAseqDB_1001-results-2.yaml
results/human/gpt4/HALLMARK_HYPOXIA-results-2.yaml
results/human/gpt4/HALLMARK_DNA_REPAIR-results-2.yaml
results/human/gpt4/HALLMARK_G2M_CHECKPOINT-results-2.yaml
results/human/gpt4/EDS-results-2.yaml
results/human/gpt4/HALLMARK_IL2_STAT5_SIGNALING-results-2.yaml
results/human/gpt4/HALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION-results-2.yaml
results/human/gpt4/HALLMARK_PI3K_AKT_MTOR_SIGNALING-results-2.yaml
results/human/gpt4/HALLMARK_COAGULATION-results-2.yaml
results/human/gpt4/peroxisome-results-2.yaml
results/human/gpt4/HALLMARK_APICAL_JUNCTION-results-2.yaml
results/human/gpt4/HALLMARK_ANGIOGENESIS-results-2.yaml
results/human/gpt4/go-postsynapse-calcium-results-2.yaml
results/human/gpt4/HALLMARK_BILE_ACID_METABOLISM-results-2.yaml
results/human/gpt4/HALLMARK_CHOLESTEROL_HOMEOSTASIS-results-2.yaml
results/human/gpt4/bicluster_RNAseqDB_1002-results-2.yaml
results/huma

In [9]:
len(comps)

144

In [10]:
results_by_gene_sets = comps

## Create the main data frame

For each gene set, we will create a list of rows,
for each combination of method + parameters, plus a cutoff.

A cutoff of zero means we only consider the *top rank* GO term from enrichment.
If the results contain this, it's a true positive; otherwise we count as a single false positive.

Otherwise we look at all enrichment results with p-val less than the cutoff. If predicted terms match these,
it's a true positive.

For calculating false negatives, we check both ancestors and descendants

In [11]:
import itertools
pairs = list(itertools.product([0.005, 0.05, 99], [False, True], [1, 5, 10, 25, 5000]))

def eval_gene_set_result(gs):
    """
    For each gene set makes, sets of rows, for each combination of method/params,
    plus each combination of p-value cutoff
    """
    rows = []
    std = gs["payloads"]["standard"]
    if "enrichment_results" not in std:
        print(f"NO GOLD STANDARD: {gs['name']}")
    expected_results = std.get("enrichment_results", [])
    for run, method_result in gs["payloads"].items():
        predicted_term_ids = method_result.get("term_ids", [])

        #closure_map = defaultdict(list)
        predicted_term_closure_ids = set()
        for t in predicted_term_ids:
            predicted_term_closure_ids.update(closure_map.get(t, set()))
        #for s, _, o in go.relationships(subjects=predicted_term_ids, predicates=[IS_A, PART_OF], include_entailed=True):
            #closure_map[s].append(o)
        #    predicted_term_closure_ids.add(o)
        for cutoff, use_closure, top_n in pairs:
            #if top_n == 1 and cutoff != 0.05:
            #    continue
            row = {}

            ###previously this was from:
            ##go_term_ids = [t for t in payload.term_ids if t.startswith("GO:")]
            row['go_term_ids'] = predicted_term_closure_ids#gs["payloads"]["closure"]["term_ids"]#go_term_ids
            row["name"] = f"{gs['name']}-{cutoff}"
            row["cutoff"] = cutoff
            row["closure"] = use_closure
            row["top_n"] = top_n
            method = method_result.get("method", "")
            approach = "gpt"
            if method == "no_synopsis":
                src = "NONE"
            elif method == "ontological_synopsis":
                src = "GO"
            elif method == "narrative_synopsis":
                src = "RefSeq"
            else:
                approach = method
                src = ""
            row["source"] = src
            model = method_result.get("model", "")
            if model == "gpt-4":
                model = "4"
            elif model == "gpt-3.5-turbo":
                model = "3.5"
            elif model == "text-davinci-003":
                model = "3"
            row["model"] = model
            row["method"] = approach
            if model:
                row["method_desc"] = f"{method}-{model}"
            else:
                row["method_desc"] = method_result.get("truncation_factor", "")
            row["run"] = run
            for k in ["truncation_factor", "prompt_variant", "response_token_length"]:
                row[k] = method_result.get(k, "")
            row["prompt_length"] = len(method_result.get("prompt", ""))
            true_positive_terms = []
            false_negative_terms = []
            more_specific_false_negative_terms = []   # predicted a descendant
            more_general_false_negative_terms = []   # predicted a descendant
            unparsed_terms = []
            standard_enrichment_results = [(r["p_value_adjusted"], r["class_id"]) for r in expected_results]
            enrichment_closure = set()
            enrichment_term_ids = set()
            if cutoff == 99:
                # extend the enrichment results but with max cutoff
                for x in gs["payloads"]["closure"]["term_ids"]:
                    standard_enrichment_results.append((cutoff, x))
            visited_terms = set()
            n = 0
            for p_val, true_term_id in standard_enrichment_results:
                if cutoff > 0 and p_val > cutoff:
                    break
                if true_term_id in visited_terms:
                    continue
                n += 1
                if n > top_n:
                    break
                enrichment_closure.update(closure_map.get(true_term_id, set()))
                enrichment_term_ids.add(true_term_id)
                true_term_closure_ids = closure_map.get(true_term_id, set())
                if use_closure:
                    visited_terms.update(true_term_closure_ids)
                else:
                    visited_terms.add(true_term_id)
                if true_term_id in predicted_term_ids:
                    true_positive_terms.append(true_term_id)
                elif true_term_id in predicted_term_closure_ids:
                    # predicted a more specific term
                    if use_closure:
                        true_positive_terms.append(true_term_id)
                    else:
                        false_negative_terms.append(true_term_id)
                    more_specific_false_negative_terms.append(true_term_id)
                elif true_term_closure_ids.intersection(predicted_term_ids):
                    # predicted a more general term
                    if use_closure:
                        true_positive_terms.append(true_term_id)
                    else:
                        false_negative_terms.append(true_term_id)
                    more_general_false_negative_terms.append(true_term_id)
                else:
                    false_negative_terms.append(true_term_id)
                if cutoff == 0:
                    break
            false_positive_terms = []
            if top_n > 1:
                for t in predicted_term_ids:
                    if t not in true_positive_terms:
                        if t.startswith("GO:"):
                            if not use_closure:
                                false_positive_terms.append(t)
                            else:
                                if t in enrichment_closure:
                                    # prediction is more general
                                    pass
                                elif closure_map.get(t, set()).intersection(enrichment_term_ids):
                                    # prediction is more specific
                                    pass
                                else:
                                    false_positive_terms.append(t)
                        elif ":" in t:
                            # MONDO, UBERON, etc
                            pass
                        else:
                            unparsed_terms.append(t)
            else:
                if not true_positive_terms:
                    if predicted_term_ids:
                        false_positive_terms.append(predicted_term_ids[0])
            row["true_positives"] = len(true_positive_terms)
            row["false_positives"] = len(false_positive_terms)
            row["false_negatives"] = len(false_negative_terms)
            row["more_general_false_negatives"] = len(more_general_false_negative_terms)
            row["more_specific_false_negatives"] = len(more_specific_false_negative_terms)
            row["all_predictions_closure"] = len(predicted_term_closure_ids)
            row["unparsed"] = len(unparsed_terms)
            row["true_positive_terms"] = "|".join(true_positive_terms)
            row["false_positive_terms"] = "|".join(false_positive_terms)
            row["false_negative_terms_example20"] = "|".join(false_negative_terms)[0:20]
            row["unparsed_terms"] = "|".join(unparsed_terms)
            row["gene_set_size"] = len(gs.get("gene_symbols"))
            
            denom1 = row["true_positives"] + row["false_positives"]
            if denom1 != 0:
                row["precision"] = row["true_positives"] / denom1
            else:
                row["precision"] = 0

            denom2 = row["true_positives"] + row["false_negatives"]
            if denom2 != 0:
                row["recall"] = row["true_positives"] / denom2
            else:
                row["recall"] = 0
                
            denom3 = row["true_positives"] + row["more_general_false_negatives"]
            if denom3 != 0:
                row["recall_general"] = row["true_positives"] / denom3
            else:
                row["recall_general"] = 0
                
            denom4 = row["true_positives"] + row["more_specific_false_negatives"]
            if denom4 != 0:
                row["recall_specific"] = row["true_positives"] / denom4
            else:
                row["recall_specific"] = 0
            
            f1_denom = row["true_positives"] + 0.5* (row["false_positives"] + row["false_negatives"]) 
            if(f1_denom != 0):
                row["f1_score"] = row["true_positives"] / (f1_denom)
            else:
                row["f1_score"] = 0

            #mcc_denom = math.sqrt((row["true_positives"] + row["false_positives"]) * (row["true_positives"] + row["false_negatives"]) * (row["true_negatives"] + row["false_positives"]) * (row["true_negatives"] + row["false_negatives"]))
            #mcc_num = (row["true_negatives"] * row["true_positives"]) -  (row["false_negatives"] * row["false_positives"])
            #if(f1_denom != 0):
            #    row["mcc"] =  mcc_num / (mcc_denom)
            #else:
            #    row["mcc"] = None

            #row["gene_symbols"] = "|".join(gs.get("gene_symbols", []))
            #row["gene_ids"] = "|".join(gs.get("gene_ids", []))
            rows.append(row)
    return rows

rows = eval_gene_set_result(results_by_gene_sets[4])
df = pd.DataFrame(rows)
df.to_csv("results/TEST-processed.tsv", sep="\t", index=False)
df

,go_term_ids,name,cutoff,closure,top_n,source,model,method,method_desc,run,...,true_positive_terms,false_positive_terms,false_negative_terms_example20,unparsed_terms,gene_set_size,precision,recall,recall_general,recall_specific,f1_score
0,"{GO:0006810, GO:0032502, GO:0072359, GO:000998...",HALLMARK_HYPOXIA-0-0.005,0.005,False,1,NONE,4,gpt,no_synopsis-4,gpt-4.no_synopsis.v1,...,GO:0005975,,,,200,1.000000,1.000000,1.000000,1.000000,1.000000
1,"{GO:0006810, GO:0032502, GO:0072359, GO:000998...",HALLMARK_HYPOXIA-0-0.005,0.005,False,5,NONE,4,gpt,no_synopsis-4,gpt-4.no_synopsis.v1,...,GO:0005975|GO:0006096,GO:1904659|GO:0051726|GO:0001525|GO:0030198,GO:0005996|GO:001931,oxidative stress response|apoptosis regulation...,200,0.333333,0.400000,0.500000,0.666667,0.363636
2,"{GO:0006810, GO:0032502, GO:0072359, GO:000998...",HALLMARK_HYPOXIA-0-0.005,0.005,False,10,NONE,4,gpt,no_synopsis-4,gpt-4.no_synopsis.v1,...,GO:0005975|GO:0006096,GO:1904659|GO:0051726|GO:0001525|GO:0030198,GO:0005996|GO:001931,oxidative stress response|apoptosis regulation...,200,0.333333,0.200000,0.333333,0.500000,0.250000
3,"{GO:0006810, GO:0032502, GO:0072359, GO:000998...",HALLMARK_HYPOXIA-0-0.005,0.005,False,25,NONE,4,gpt,no_synopsis-4,gpt-4.no_synopsis.v1,...,GO:0005975|GO:0006096,GO:1904659|GO:0051726|GO:0001525|GO:0030198,GO:0005996|GO:001931,oxidative stress response|apoptosis regulation...,200,0.333333,0.080000,0.133333,0.400000,0.129032
4,"{GO:0006810, GO:0032502, GO:0072359, GO:000998...",HALLMARK_HYPOXIA-0-0.005,0.005,False,5000,NONE,4,gpt,no_synopsis-4,gpt-4.no_synopsis.v1,...,GO:0005975|GO:0006096|GO:0001525,GO:1904659|GO:0051726|GO:0030198,GO:0005996|GO:001931,oxidative stress response|apoptosis regulation...,200,0.500000,0.013453,0.125000,0.096774,0.026201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,"{GO:0005575, GO:0061311, GO:0034139, GO:009897...",HALLMARK_HYPOXIA-0-99,99.000,True,1,,,,,closure,...,GO:0005975,,,,200,1.000000,1.000000,1.000000,1.000000,1.000000
686,"{GO:0005575, GO:0061311, GO:0034139, GO:009897...",HALLMARK_HYPOXIA-0-99,99.000,True,5,,,,,closure,...,GO:0005975|GO:0005996|GO:0019318|GO:0016052|GO...,GO:0050930|GO:0006361|GO:0030145|GO:0006011|GO...,,,200,0.000616,1.000000,1.000000,1.000000,0.001231
687,"{GO:0005575, GO:0061311, GO:0034139, GO:009897...",HALLMARK_HYPOXIA-0-99,99.000,True,10,,,,,closure,...,GO:0005975|GO:0005996|GO:0019318|GO:0016052|GO...,GO:0050930|GO:0006361|GO:0030145|GO:0006011|GO...,,,200,0.001235,1.000000,1.000000,1.000000,0.002466
688,"{GO:0005575, GO:0061311, GO:0034139, GO:009897...",HALLMARK_HYPOXIA-0-99,99.000,True,25,,,,,closure,...,GO:0005975|GO:0005996|GO:0019318|GO:0016052|GO...,GO:0050930|GO:0006361|GO:0030145|GO:0061975|GO...,,,200,0.003627,1.000000,1.000000,1.000000,0.007229


In [12]:
rows = []
for gs in results_by_gene_sets:
    this_rows = eval_gene_set_result(gs)
    rows.extend(this_rows)
    print(len(rows))
    #df = pd.DataFrame(rows)
    #df.to_csv("results/TEMP.tsv", sep="\t", index=False)
    
    #if len(rows) % 10 == 0:
    #    print(f"Processed {len(rows)})
              
df = pd.DataFrame(rows)
pd.set_option('display.max_rows', 10)
df.to_csv("results/processed.tsv", sep="\t", index=False)
df

690
1380
NO GOLD STANDARD: bicluster_RNAseqDB_1001-0
2070
NO GOLD STANDARD: bicluster_RNAseqDB_1001-1
2760
3450
4140
4830
5520
6210
6900
7590
8280
8970
9660
10350
11040
11730
12420
13110
13800
14490
15180
15870
16560
17250
17940
18630
19320
20010
20700
21390
22080
22770
23460
24150
24840
25530
26220
26910
27600
28290
28980
29670
30360
31050
31740
32430
33120
33810
34500
35190
35880
36570
37260
37950
38640
39330
40020
40710
41400
42090
42780
43470
44160
44850
45540
46230
46920
47610
48300
48990
49680
50370
51060
51750
52440
53130
53820
54510
55200
55890
56580
57270
57960
58650
59340
60030
60720
61410
62100
62790
63480
64170
64860
65550
66240
66930
67620
68310
69000
69690
70380
71070
71760
72450
73140
73830
74520
75210
75900
76590
77280
77970
78660
79350
80040
80730
81420
82110
82800
83490
84180
84870
85560
86250
86940
87630
88320
89010
89700
90390
91080
91770
92460
93150
93840
94530
95220
95910
96600
97290
97980
98670
99360


,go_term_ids,name,cutoff,closure,top_n,source,model,method,method_desc,run,...,true_positive_terms,false_positive_terms,false_negative_terms_example20,unparsed_terms,gene_set_size,precision,recall,recall_general,recall_specific,f1_score
0,"{GO:0044237, GO:0009987, GO:0006082, GO:000599...",glycolysis-gocam-0-0.005,0.005,False,1,NONE,4,gpt,no_synopsis-4,gpt-4.no_synopsis.v1,...,GO:0006096,,,,10,1.000000,1.000000,1.00,1.000000,1.000000
1,"{GO:0044237, GO:0009987, GO:0006082, GO:000599...",glycolysis-gocam-0-0.005,0.005,False,5,NONE,4,gpt,no_synopsis-4,gpt-4.no_synopsis.v1,...,GO:0006096,GO:0006006,GO:0006090|GO:001605,energy production|atp generation,10,0.500000,0.200000,1.00,0.200000,0.285714
2,"{GO:0044237, GO:0009987, GO:0006082, GO:000599...",glycolysis-gocam-0-0.005,0.005,False,10,NONE,4,gpt,no_synopsis-4,gpt-4.no_synopsis.v1,...,GO:0006096,GO:0006006,GO:0006090|GO:001605,energy production|atp generation,10,0.500000,0.100000,0.20,0.200000,0.166667
3,"{GO:0044237, GO:0009987, GO:0006082, GO:000599...",glycolysis-gocam-0-0.005,0.005,False,25,NONE,4,gpt,no_synopsis-4,gpt-4.no_synopsis.v1,...,GO:0006096|GO:0006006,,GO:0006090|GO:001605,energy production|atp generation,10,1.000000,0.080000,0.25,0.153846,0.148148
4,"{GO:0044237, GO:0009987, GO:0006082, GO:000599...",glycolysis-gocam-0-0.005,0.005,False,5000,NONE,4,gpt,no_synopsis-4,gpt-4.no_synopsis.v1,...,GO:0006096|GO:0006006,,GO:0006090|GO:001605,energy production|atp generation,10,1.000000,0.051282,0.25,0.142857,0.097561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99355,"{GO:0005575, GO:0030880, GO:0018030, GO:012001...",HALLMARK_MTORC1_SIGNALING-1-99,99.000,True,1,,,,,closure,...,GO:0044283,,,,180,1.000000,1.000000,1.00,1.000000,1.000000
99356,"{GO:0005575, GO:0030880, GO:0018030, GO:012001...",HALLMARK_MTORC1_SIGNALING-1-99,99.000,True,5,,,,,closure,...,GO:0044283|GO:0005737|GO:0019752|GO:0036094|GO...,GO:0008525|GO:0009749|GO:0019899|GO:0050709|GO...,,,180,0.000794,1.000000,1.00,1.000000,0.001587
99357,"{GO:0005575, GO:0030880, GO:0018030, GO:012001...",HALLMARK_MTORC1_SIGNALING-1-99,99.000,True,10,,,,,closure,...,GO:0044283|GO:0005737|GO:0019752|GO:0036094|GO...,GO:0008525|GO:0009749|GO:0019899|GO:0050709|GO...,,,180,0.001720,1.000000,1.00,1.000000,0.003434
99358,"{GO:0005575, GO:0030880, GO:0018030, GO:012001...",HALLMARK_MTORC1_SIGNALING-1-99,99.000,True,25,,,,,closure,...,GO:0044283|GO:0005737|GO:0019752|GO:0036094|GO...,GO:0008525|GO:0019899|GO:0050709|GO:0035094|GO...,,,180,0.004761,1.000000,1.00,1.000000,0.009477


In [13]:
print(df.shape)
#df[[model, method]].drop_duplicates()
print(df.shape)

(99360, 31)
(99360, 31)


In [20]:
#df.to_csv("results/processed.tsv", sep="\t", index=False, lineterminator='\r\n')

In [24]:
new_df = df.replace({'\n': ' ', '\r': ' '}, regex=True)
new_df.to_csv("results/processed.tsv", sep="\t", index=False)

In [14]:
df["method"].unique()

array(['gpt', 'standard', 'standard_no_ontology', 'random', 'rank_based',
       ''], dtype=object)